<a href="https://colab.research.google.com/github/ngurahgotama/i-gusti-ngurah-gotama-web-scraping-assignment/blob/main/scrape_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pendahuluan

Dalam era digital ini, data menjadi salah satu aset terpenting bagi berbagai jenis organisasi dan perusahaan. Dengan kemampuan untuk mengakses dan menganalisis data, kita dapat memperoleh wawasan yang berharga untuk mendukung pengambilan keputusan yang lebih baik.

Dalam proyek ini, kami akan menjelajahi web scraping, teknik yang memungkinkan kami untuk mengekstrak informasi dari halaman web dan menyimpannya dalam format yang dapat diakses dan dianalisis. Situs web yang kami pilih untuk demonstrasi ini adalah https://books.toscrape.com/, sebuah toko buku online yang menyediakan berbagai judul buku dari berbagai kategori.

Tujuan dari proyek ini adalah untuk mengumpulkan informasi tentang buku-buku yang tersedia di situs web tersebut, termasuk judul, penulis, harga, dan ulasan pengguna. Kami akan menggunakan bahasa pemrograman Python dan pustaka Beautiful Soup untuk melakukan scraping data dari halaman web tersebut.

In [12]:
#Mengimpor modul-modul yang diperlukan
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
url = "https://books.toscrape.com/catalogue/category/books/travel_2/index.html"

page = requests.get(url)
page.headers

{'Date': 'Thu, 08 Feb 2024 07:04:07 GMT', 'Content-Type': 'text/html', 'Content-Length': '37009', 'Connection': 'keep-alive', 'Last-Modified': 'Wed, 08 Feb 2023 21:02:32 GMT', 'ETag': '"63e40de8-9091"', 'Accept-Ranges': 'bytes', 'Strict-Transport-Security': 'max-age=0; includeSubDomains; preload'}

In [2]:
page.text

'\n\n<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    Travel | \n     Books to Scrape - Sandbox\n\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="\n    \n" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shor

In [3]:
type(page.text)

str

In [4]:
type(page.content)

bytes

In [6]:
# print(type(page.content))

soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
type(soup)

bs4.BeautifulSoup

In [8]:
products = soup.find_all("article", class_="product_pod") # find_all akan mengembalikan list (array)
type(products)

bs4.element.ResultSet

In [9]:
names = []
prices = []
ratings = []

In [11]:
rating_pattern = re.compile("star-rating.*")

def get_rating(txt: str)-> int:
  mapper = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5}
  return mapper[txt]

In [13]:
for product in products:
  product_name_tag = product.find("h3") # dapet nama
  product_price_tag = product.find("div", class_="product_price")
  product_price_txt = product_price_tag.find("p", class_="price_color").text
  rating_tag = product.find("p", class_=rating_pattern)
  rating_num = get_rating(rating_tag['class'][1].lower())

  # Bikin list
  names.append(product_name_tag.text)
  prices.append(product_price_txt)
  ratings.append(rating_num)

df = pd.DataFrame({
    "names": names,
    "prices": prices,
    "ratings": ratings
})

In [14]:
# Menampilkan dataframe
df

,names,prices,ratings
0,It's Only the Himalayas,£45.17,2
1,Full Moon over Noah’s ...,£49.43,4
2,See America: A Celebration ...,£48.87,3
3,Vagabonding: An Uncommon Guide ...,£36.94,2
4,Under the Tuscan Sun,£37.33,3
5,A Summer In Europe,£44.34,2
6,The Great Railway Bazaar,£30.54,1
7,A Year in Provence ...,£56.88,4
8,The Road to Little ...,£23.21,1
9,Neither Here nor There: ...,£38.95,3


In [15]:
# Simpan DataFrame ke dalam file CSV
df.to_csv("books.csv", index=False)